<a href="https://www.kaggle.com/code/avtnshm/movies-reco-using-genre-extraction-from-overiews?scriptVersionId=157306140" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/10000-movies-dataset-with-rating/movies.csv


In [2]:
df = pd.read_csv('/kaggle/input/10000-movies-dataset-with-rating/movies.csv')
df.head()

,Unnamed: 0,id,title,overview,release_date,popularity,vote_average,vote_count
0,0,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,95.013,8.7,18651
1,1,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,90.970,8.7,24618
2,2,240,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,62.918,8.6,11274
3,3,424,Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,63.161,8.6,14561
4,4,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",1995-10-20,32.221,8.6,4247


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9980 entries, 0 to 9979
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    9980 non-null   int64  
 1   id            9980 non-null   int64  
 2   title         9980 non-null   object 
 3   overview      9978 non-null   object 
 4   release_date  9980 non-null   object 
 5   popularity    9980 non-null   float64
 6   vote_average  9980 non-null   float64
 7   vote_count    9980 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 623.9+ KB


In [4]:
df.isnull().sum()

Unnamed: 0      0
id              0
title           0
overview        2
release_date    0
popularity      0
vote_average    0
vote_count      0
dtype: int64

In [5]:
df.dropna(inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
import datetime
df['release_date'] = pd.to_datetime(df['release_date'])

df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

df.drop(columns=['release_date'], inplace=True)

In [7]:
df.describe()

,id,popularity,vote_average,vote_count,release_year,release_month
count,9.978000e+03,9978.000000,9978.000000,9978.000000,9978.000000,9978.000000
mean,1.858227e+05,24.299962,6.784085,1689.602024,2002.873622,7.056224
std,2.433763e+05,53.994726,0.628551,2914.224792,18.469644,3.419391
min,2.000000e+00,0.600000,5.700000,200.000000,1895.000000,1.000000
25%,1.035150e+04,12.665250,6.300000,323.000000,1995.000000,4.000000
50%,3.555300e+04,17.117000,6.700000,619.000000,2009.000000,8.000000
75%,3.487432e+05,24.820000,7.200000,1609.000000,2016.000000,10.000000
max,1.140066e+06,2292.177000,8.700000,34461.000000,2023.000000,12.000000


In [8]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df[['id', 'popularity', 'vote_average', 'vote_count', 'release_year', 'release_month']]
y = df['vote_average']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostRegressor(iterations=500, depth=10, learning_rate=0.05, loss_function='RMSE')
model.fit(X_train, y_train, cat_features=['id', 'release_year', 'release_month'])

y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)


0:	learn: 0.5978913	total: 63ms	remaining: 31.4s
1:	learn: 0.5694220	total: 79ms	remaining: 19.7s
2:	learn: 0.5431109	total: 106ms	remaining: 17.5s
3:	learn: 0.5177013	total: 122ms	remaining: 15.1s
4:	learn: 0.4945967	total: 131ms	remaining: 13s
5:	learn: 0.4715327	total: 159ms	remaining: 13.1s
6:	learn: 0.4504638	total: 169ms	remaining: 11.9s
7:	learn: 0.4306488	total: 179ms	remaining: 11s
8:	learn: 0.4110247	total: 193ms	remaining: 10.5s
9:	learn: 0.3920993	total: 214ms	remaining: 10.5s
10:	learn: 0.3739637	total: 239ms	remaining: 10.6s
11:	learn: 0.3575909	total: 246ms	remaining: 9.98s
12:	learn: 0.3412996	total: 270ms	remaining: 10.1s
13:	learn: 0.3252489	total: 281ms	remaining: 9.74s
14:	learn: 0.3113194	total: 302ms	remaining: 9.77s
15:	learn: 0.2973255	total: 323ms	remaining: 9.76s
16:	learn: 0.2839097	total: 334ms	remaining: 9.48s
17:	learn: 0.2706369	total: 354ms	remaining: 9.48s
18:	learn: 0.2585301	total: 373ms	remaining: 9.45s
19:	learn: 0.2472237	total: 395ms	remaining: 9.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9978 entries, 0 to 9979
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             9978 non-null   int64  
 1   title          9978 non-null   object 
 2   overview       9978 non-null   object 
 3   popularity     9978 non-null   float64
 4   vote_average   9978 non-null   float64
 5   vote_count     9978 non-null   int64  
 6   release_year   9978 non-null   int32  
 7   release_month  9978 non-null   int32  
dtypes: float64(2), int32(2), int64(2), object(2)
memory usage: 623.6+ KB


In [10]:
genre_keywords = {
    'Action': ['action', 'fight', 'thrilling', 'combat', 'adventure'],
    'Romance': ['romance', 'love', 'heartfelt', 'relationship', 'passion'],
    'Comedy': ['comedy', 'funny', 'humor', 'laugh', 'hilarious'],
    'Drama': ['drama', 'emotional', 'intense', 'tragic', 'heartbreaking'],
    'Sci-Fi': ['sci-fi', 'space', 'futuristic', 'alien', 'technology'],
}

def extract_genre_from_overview(overview):
    overview_lower = overview.lower()
    for genre, keywords in genre_keywords.items():
        if any(keyword in overview_lower for keyword in keywords):
            return genre
    return 'Unknown'  
df['extracted_genre'] = df['overview'].apply(extract_genre_from_overview)
df[['title', 'overview', 'extracted_genre']]


,title,overview,extracted_genre
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",Unknown
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,Romance
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,Unknown
3,Schindler's List,The true story of how businessman Oskar Schind...,Unknown
4,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",Romance
...,...,...,...
9975,The Messengers,When the Solomons trade in the craziness of bi...,Unknown
9976,Let Us Prey,"Rachel, a rookie cop, is about to begin her fi...",Unknown
9977,The Whole Ten Yards,"Jimmy ""The Tulip"" Tudeski now spends his days ...",Unknown
9978,Jean-Philippe,A bourgeois office drone whose raison d’état i...,Unknown


In [11]:
unknown_count = df['extracted_genre'].value_counts().get('Unknown', 0)
print(f"Number of movies with unknown genre: {unknown_count}")

Number of movies with unknown genre: 7190


In [12]:
df = df[df['extracted_genre'] != 'Unknown']

In [16]:
genre_options = {
    1: 'Action',
    2: 'Comedy',
    3: 'Drama',
    4: 'Science Fiction',
    5: 'Thriller'
}

print("Choose a genre:")
for key, value in genre_options.items():
    print(f"{key}. {value}")


selected_genre = int(input("Enter the number corresponding to your preferred genre: "))

if selected_genre not in genre_options:
    print("Invalid input. Please choose a valid number.")
else:
    selected_genre = genre_options[selected_genre]
    print(f"You selected '{selected_genre}' as your preferred genre.")

Choose a genre:
1. Action
2. Comedy
3. Drama
4. Science Fiction
5. Thriller


Enter the number corresponding to your preferred genre:  1


You selected 'Action' as your preferred genre.


In [17]:
genre_movies = df[df['extracted_genre'] == selected_genre]
sorted_movies = genre_movies.sort_values(by='vote_average', ascending=False)

top_20_recommendations = sorted_movies.head(20)
print("Top 20 Recommendations:")
print(top_20_recommendations[['title', 'vote_average']])

user_input = input("Type 'next' if you want to see more recommendations, or press Enter to exit: ")

if user_input.lower() == 'next':
    next_30_recommendations = sorted_movies.iloc[20:50]  
    print("\nNext 30 Recommendations:")
    print(next_30_recommendations[['title', 'vote_average']])
    print("\nThanks for using the recommendation system!")
else:
    print("Thanks for using the recommendation system!")

Top 20 Recommendations:
                                         title  vote_average
11                                Pulp Fiction           8.5
34                                Interstellar           8.4
41                        Howl's Moving Castle           8.4
50                          Hotarubi no Mori e           8.4
53                          American History X           8.4
23                                  Fight Club           8.4
89                Puss in Boots: The Last Wish           8.3
91                          The Legend of 1900           8.3
93                           Avengers: Endgame           8.3
80                             Five Feet Apart           8.3
70                                Come and See           8.3
59                           Princess Mononoke           8.3
128                                 The Matrix           8.2
149                           Jujutsu Kaisen 0           8.2
154                                       Togo           8.2


Type 'next' if you want to see more recommendations, or press Enter to exit:  next



Next 30 Recommendations:
                                                 title  vote_average
237                                             WALL·E           8.1
225        Scooby-Doo! and KISS: Rock and Roll Mystery           8.1
201                                           Rashomon           8.1
209                Along with the Gods: The Two Worlds           8.1
197                                    Innocent Voices           8.1
189                                  Full Metal Jacket           8.1
342                 Kill Bill: The Whole Bloody Affair           8.0
380                              The Cranes Are Flying           8.0
364        Scooby-Doo! and the Curse of the 13th Ghost           8.0
351                 Tae Guk Gi: The Brotherhood of War           8.0
322                                 No Manches Frida 2           8.0
314                              The Boy and the Beast           8.0
294                                       Citizen Kane           8.0
282     